# EPA1361 - Model-Based Decision Making
# Week 3 - Sensitivity analysis

This exercise uses the same predator-prey model we used for the multi-model exercise, focusing on the Python version.
As with the other exercise, define a model object for the function below, with the uncertainty ranges provided:

|Parameter	|Range or value	        |
|-----------|--------------:|
|prey_birth_rate    	|0.015 – 0.035	|
|predation_rate|0.0005 – 0.003 	|
|predator_efficiency     	|0.001 – 0.004	    |
|predator_loss_rate	    |0.04 – 0.08	    |

* Sensitivity analysis often focuses on the final values of an outcome at the end of the simulation.
However, we can also look at metrics that give us additional information about the behavior of the model over time.
Using [the statsmodel library](https://www.statsmodels.org/stable/index.html) and an appropriate sampling design,
fit a linear regression model for each of the following indicators. What can we conclude about the behavior of the model,
and about the importance of the different inputs?

  * The final values of the _prey_ outcome
  * The mean values of the _prey_ outcome over time, within each experiment
  * The standard deviations of the _prey_ outcome over time, within each experiment
  

* Use the Sobol sampling functionality included in the Workbench to perform experiments with a sample size of N=50,
then analyze the results with SALib for the same three indicators. This requires specifying the keyword argument `'uncertainty_sampling'`
of perform_experiments.
Note that when using Sobol sampling, the meaning of the keyword argument `scenarios` changes a bit.
In order to properly estimate Sobol scores as well as interaction effects, you require N * (2D+2) scenarios,
where D is the number of uncertain parameters, and N is the value for scenarios passed to `perform_experiments`.
Repeat the analysis for larger sample sizes, with N=250 and N=1000. How can we interpret the first-order and total indices?
Are these sample sizes sufficient for a stable estimation of the indices?
You'll need to use the [get_SALib_problem](https://emaworkbench.readthedocs.io/en/latest/ema_documentation/em_framework/salib_samplers.html)
function to convert your Workbench experiments to a problem definition that you can pass to the SALib analysis function.

* *hint*: sobol is a deterministic sequence of quasi random numbers. Thus, you can run with N=1000 and simply slice for 1:50 and 1:250.

* Use the [Extra-Trees analysis](https://emaworkbench.readthedocs.io/en/latest/ema_documentation/analysis/feature_scoring.html)
included in the Workbench to approximate the Sobol total indices, with a suitable sampling design. As a starting point,
use an ensemble of 100 trees and a max_features parameter of 0.6, and set the analysis to regression mode. Are the estimated importances stable relative to the sample size and the analysis parameters? How do the results compare to the Sobol indices? For more details on this analysis see [Jaxa-Rozen & Kwakkel (2018)](https://www.sciencedirect.com/science/article/pii/S1364815217311581)

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from patsy.highlevel import dmatrices

from ema_workbench import (Model, RealParameter, TimeSeriesOutcome, ScalarOutcome, perform_experiments, ema_logging, SequentialEvaluator)
from ema_workbench.analysis import plotting, plotting_util
from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS

from ema_workbench.analysis import feature_scoring
from ema_workbench.analysis.scenario_discovery_util import RuleInductionType
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from SALib.analyze import sobol

# For every dependent variable (final, mean, std) we
def fit_linear_regression(x,Y)

def pred_prey(prey_birth_rate=0.025, predation_rate=0.0015, predator_efficiency=0.002,
             predator_loss_rate=0.06, initial_prey=50, initial_predators=20, dt=0.25, 
             final_time=365, reps=1):

    #Initial values
    predators = np.zeros((reps, int(final_time/dt)+1))
    prey = np.zeros((reps, int(final_time/dt)+1))
    sim_time = np.zeros((reps, int(final_time/dt)+1))
    
    for r in range(reps):

        predators[r,0] = initial_predators
        prey[r,0] = initial_prey

    #Calculate the time series
    for t in range(0, sim_time.shape[1]-1):

        dx = (prey_birth_rate*prey[r,t]) - (predation_rate*prey[r,t]*predators[r,t])
        dy = (predator_efficiency*predators[r,t]*prey[r,t]) - (predator_loss_rate*predators[r,t])

        prey[r,t+1] = max(prey[r,t] + dx*dt, 0)
        predators[r,t+1] = max(predators[r,t] + dy*dt, 0)
        sim_time[r,t+1] = (t+1)*dt
    
    #Return outcomes
    return {'TIME':sim_time,
            'predators':predators,
            'prey':prey}

# Instantiate Python model
modelPython = Model('PredPreyPython', function=pred_prey) # instantiate the model

# Function expects an instance of a model and plots it
def plot_lotka_volterra(model_instance):

    model_instance.uncertainties = [RealParameter ('prey_birth_rate', 0.015, 0.035),
                                 RealParameter ('predation_rate', 0.0005, 0.003),
                                 RealParameter ('predator_efficiency', 0.001, 0.004),
                                 RealParameter ('predator_loss_rate', 0.04, 0.08)
                                 ]

    model_instance.outcomes = [TimeSeriesOutcome('TIME'),
                            TimeSeriesOutcome('predators'),
                            TimeSeriesOutcome('prey')]

    with SequentialEvaluator(model_instance) as evaluator:
        sa_results = evaluator.perform_experiments(scenarios=5, uncertainty_sampling=LHS)

    experiments, outcomes = sa_results

    # Squeeze outcomes
    outcomes_squeezed = {}

    for key in outcomes:
        outcomes_squeezed[key] = np.squeeze(outcomes[key])

    # Store final values of prey outcome
    prey_final = []
    prey_mean = []
    prey_std = []

    # Iterate through rows of outcome numpy array (experiments)
    for experiment_row in outcomes_squeezed["prey"]:
        prey_final.append(experiment_row[-1]) # Get last element
        prey_mean.append(np.mean(experiment_row))
        prey_std.append(np.std(experiment_row))

    predation_rate = experiments["predation_rate"]

    #for outcome_key in outcomes_squeezed.keys():
    #   if outcome_key != 'TIME':
            # Fetch numpy array from dictionary, and calculate average over every single column (average at every point of time)
            # Plot using workbench
            #plotting.lines(experiments, outcomes_squeezed, outcomes_to_show=outcome_key, density=plotting_util.Density.HIST)

    return prey_final, prey_mean, prey_std

    # problem = get_SALib_problem(model_instance.uncertainties)
    # Si = sobol.analyze(problem, outcomes['prey'], calc_second_order=True, print_to_console=False)
    #
    # scores_filtered = {k:Si[k] for k in ['ST','ST_conf','S1','S1_conf']}
    # Si_df = pd.DataFrame(scores_filtered, index=problem['names'])
    #
    # sns.set_style('white')
    # fig, ax = plt.subplots(1)
    #
    # indices = Si_df[['S1','ST']]
    # err = Si_df[['S1_conf','ST_conf']]
    #
    # indices.plot.bar(yerr=err.values.T,ax=ax)
    # fig.set_size_inches(8,6)
    # fig.subplots_adjust(bottom=0.3)
    # plt.show()

    # Thoughts for the error calculation:
    # For every key (predators, prey), at every time step, calculate the average over all scenarios
    # Then, we have a list of averages for all time steps
    # Then, calculate the difference between the list from Excel and the list from Python.
    # Then, find the maximum deviation. This is our maximum absolute error.



prey_final, prey_mean, prey_std = plot_lotka_volterra(modelPython)
print(prey_final)
print(prey_mean)
print(prey_std)

c:\users\marko\appdata\local\programs\python\python38\lib\site-packages\SALib\sample\saltelli.py:52: FutureWarning: 
        Convergence properties of the Sobol' sequence is only valid if `N` = 2^n.
        SALib will continue on, but results may have issues.
        In future, this will raise an error.
        
  warnings.warn(msg, FutureWarning)


[48.10571577152093, 46.989806654679086, 8.160789018409133, 19.934538923324904, 58.66325759851174, 2.5038828954993337, 48.93731589974384, 20.97866058030215, 3.0330266098251872, 4.703223314178808, 12.673062044637192, 5.544680387090113, 51.615271224706866, 4.336203978789758, 17.399357187789327, 55.51109870977218, 8.27820870654761, 51.91195312526308, 16.107287051470966, 44.74252844000247, 35.682716842237774, 38.978411852096066, 41.69387063696079, 77.1571244557235, 50.28162226689504, 51.85895950530286, 46.67239001337608, 47.787662946511226, 51.07215459956737, 46.865786646811934, 7.794703696760232, 6.809914714600715, 8.41214416247678, 17.221290599423135, 7.406671263192394, 24.217595632366343, 26.98411062513653, 8.762440437963043, 21.841016514196387, 30.516721106457695, 102.42165050783679, 59.99414988531914, 29.184953752572444, 51.54446989691301, 77.03290986189815, 3.7043395658411638, 53.06628277137696, 37.31068901154485, 5.40449731458776, 2.536074105199244]
[26.91484886907988, 27.60792966778